In [3]:
import os
from tifffile import imread, imwrite
import numpy as np

file_path = r'D:\Projects\Denosing-STED-Abberior\2022-11-03_UCF_DemoSamples\3D STED_TOM20\Average'
save_path = r'D:\Projects\Denosing-STED-Abberior\2022-11-03_UCF_DemoSamples\3D STED_TOM20\patches\Average'
list_files = os.listdir(file_path)
M = len(list_files)
patch_size = 256
patch_size_z = 16
threshold = 0.0

x1 = np.zeros((1,patch_size_z, patch_size, patch_size))


for i in list_files:
    f = imread(os.path.join(file_path, '', i)).astype(np.float64)
    f = f / f.max()
    f = f - f.min()
    f = f / f.max()
    f[f < 0] = 0
    print(f.shape)
#     f1 = np.zeros((f.shape[0],2048,2048))
#     f1[:,386:386+f.shape[1],159:159+f.shape[2]] = f
#     f = f1
    w = f.shape[-1]
    h = f.shape[-2]
    d = f.shape[0]
    if len(f.shape) == 3:
        z = f.shape[0]
    else:
        z = 1
        f = f.reshape((1, f.shape[0], f.shape[1]))
    nx = int(2*np.ceil(h / patch_size))
    ny = int(2*np.ceil(w / patch_size))
#     print(nx)
    nz = int(2*np.ceil(d/patch_size_z))
#     nx = 1
#     ny = 1
#     nz = 1
    x = np.zeros((nx * ny * nz,patch_size_z, patch_size, patch_size))
    nnx = np.floor(np.linspace(0, h - patch_size, nx)).astype(np.int32)
    nny = np.floor(np.linspace(0, w - patch_size, ny)).astype(np.int32)
    nnz = np.floor(np.linspace(0, d - patch_size_z, nz)).astype(np.int32)
    count = 0
    for j in range(nz):
        for p in range(nx):
            for q in range(ny):
                x[count, :, :,:] = f[nnz[j]:nnz[j]+patch_size_z, nnx[p]:nnx[p] + patch_size, nny[q]:nny[q] + patch_size]
                count = count + 1
#     x1 = np.append(x,x1,axis=0)

# x = x1[1::]
norm_x = np.linalg.norm(np.max(x,axis=1), axis=(1, 2))
norm_x = norm_x/ norm_x.max()
ind_norm = np.where(norm_x >= threshold)[0]
y = np.zeros((len(ind_norm),patch_size_z, patch_size, patch_size))
for k in range(len(ind_norm)):
    y[k] = x[ind_norm[k]]
    if y[k].max()>0:
        y[k] = y[k]/y[k].max()
y = np.uint16(y * (2 ** 16 - 1))
imwrite(os.path.join(save_path, '', 'Average.tif'), y, imagej=True, metadata={'axes': 'TZYX'})


(86, 1276, 1729)
(46, 873, 1010)
(20, 1060, 628)
(20, 633, 1207)
(20, 741, 941)
(20, 662, 918)
(20, 631, 659)
(20, 594, 1035)
(20, 1123, 721)
(20, 1524, 1405)
